# Тестовое задание Data Analyst | Задание 3

* Тестовое задание состоит из нескольких задач, которые необходимо реализовать на языке Python.
* Допускается применение дополнительных пакетов, будет плюсом распараллелить задачи для ускорения выполнения там где это требуется.
* Все тестовые датасеты загружаем и процессим (делаем выборки, джойны, фильтруем и т.д.) в коде.
* Если у вас не получилось сделать какие-то шаги, но вы понимаете все остальное - пропускайте их и делайте то  в чем разбираетесь.
* Результат выполнения - ссылка на github с jupiter notebook. Высылаем все ответы / ссылки на fediaeva@skytecgames.com. Так же напишите сколько часов у вас ушло на выполнение.
* При возникновении вопросов по тестовому заданию - пишем туда же, на  fediaeva@skytecgames.com
* Необходимо сделать минимум 3 задания на ваш выбор. Чем больше заданий сделали, тем лучше.
* Тестовый датасет прилагается к письму sqlite файлом `testcase.db.zip`

---

## 3 Предсказание оттока

В мобильной игре, после установки и запуска игры начинается туториал - игрок проходит обучение игре, которое состоит из нескольких последовательных шагов. 

Данные представляют собой набор строк - уникальный id игрока, datetime и номер шага туториала который пройден игроком. Геймдизайнеры выдвинули гипотезу - если мы с высокой вероятностью сможем предсказать шаг на котором игрок уйдет из игры заранее, то запустив альтернативное продолжение туториала с этого шага мы избежим оттока и увеличим прохождение туториала.

1.	Как бы вы решали задачу предсказания отвала игрока?
2.	Как оценить качество полученной модели? Опишите методы которые знаете и какой считаете оптимальным и почему.
3.	Предположим что вы разработали модель которая с достаточной вероятностью предсказывает отвал. Опишите дизайн эксперимента и методологию по проверке гипотезы геймдизайнеров

Исходные данные:

уникальный id игрока | datetime | номер шага туториала, который пройден игроком
-|-|-
... | ... | ...

<br>

Гипотезы:
* Нулевая ($ H_0 $) - Запуск альтернативного продолжения туториала с шага, на котором игрок покинет игру, не увеличит прохождение туториала
* Альтернативная ($ H_1 $) - Запуск альтернативного продолжения туториала с шага, на котором игрок покинет игру, увеличит прохождение туториала

<br>

### 3.1 Как бы вы решали задачу предсказания отвала игрока?

><br>
>Предполагается, что нам известно количество шагов в туториале.
><br><br>

<details>
  <summary style="font-size: 20px; font-weight: 500">Таблица в текстовом виде</summary>

Задача будет декомпозирована на следующие этапы:

1. Предобработка данных (проверка на наличие пропусков, аномалий, выбросов, некорректных значений и типов данных).
2. Создание дополнительных признаков: 
    * столбца с общим количеством пройденных игроком шагов туториала (`максимальное значение пройденных шагов для игрока`)
    * столбца с состоянием игрока (`ушёл из игры (1) или нет (0) = всего шагов в туториале - количество пройденных шагов, если значение > 0`)
    * столбцов с датой, месяцем, годом, часами и минутами (также можно дополнительно категоризировать по промежуткам в виде категорий: например, время дня, сезон)
3. Анализ данных (на предмет зависимости количества пройденных игроком шагов с его состоянием (ушёл или нет) от других признаков)
4. Кодирование признаков (масштабирование числовых признаков и категоризация строковых значений)
5. Отбор важных признаков (на основе анализа данных и распределение хи-квадрат или случайного леса)
6. Построение модели машинного обучения:
    * Данные: при кроссвалидации только разделение на обучающую и тестовую
    * Тип задачи: регрессия (предсказываемый шаг, на котором игрок уйдёт из игры, представляет собой непрерывное значение)
    * Целевая переменная: 
        - для игроков, покинувших игру: шаг, на котором игрок покинул игру (`< количество шагов в туториале`)
        - для игроков, оставшихся в игре: количество шагов в туториале (`== количество шагов в туториале`)
    * Метрики: 
        - MAE (`среднее абсолютных значений разностей между фактическим значением и предсказанным`)
        - процент предсказаний в пределах 1 шага от фактического значения:
            - разница между фактическим шагом отвала игрока и предсказанным значением (`фактическое значение - предсказание`):
                - если `0 или 1` - успешное предсказание
                - если `> 1 или < 1` - плохое предсказание
            - вычисление процента (`(общее количество успешных предсказаний / общее количество предсказаний) * 100%`)
    * Подбор модели (с гиперпараметрическим поиском): 
        - LinearRegression (baseline)
        - DecisionTreeRegression
        - RandomForestRegressor
        - KNeighborsRegressor
        - LGBMRegressor
        - CatBoostRegressor
    * Сравнение результатов: выбор модели с оптимальными метриками
7. Тестирование модели

</details>

№ | Этап | Описание
-|-|-
1 | **Предобработка данных** | Проверка на наличие пропусков, аномалий, выбросов, некорректных значений и типов данных |
2 | **Создание дополнительных признаков** | - Столбца с общим количеством пройденных игроком шагов туториала <br>`максимальное значение пройденных шагов для игрока` <br>- Столбца с состоянием игрока (1 или 0) <br>`ушёл или нет = всего шагов в туториале - количество пройденных шагов, если значение > 0` <br>- Столбцов с датой, месяцем, годом, часами и минутами (также можно дополнительно категоризировать по промежуткам в виде категорий:  например, время дня, сезон) |
3 | **Анализ данных** | На предмет зависимости количества пройденных игроком шагов с его состоянием (ушёл или нет) от других признаков
4 | **Кодирование признаков** | Масштабирование числовых признаков и категоризация строковых
5 | **Отбор важных признаков** | На основе анализа данных и распределение хи-квадрат или случайного леса
6 | **Построение модели машинного обучения** | |
6.1 | Данные | При кроссвалидации только разделение на обучающую и тестовую
6.2 | Тип задачи | Регрессия (предсказываемый шаг, на котором игрок уйдёт из игры, представляет собой дискретное значение)
6.3 | Целевая переменная | - Для игроков, покинувших игру: шаг, на котором игрок покинул игру <br>`< количество шагов в туториале` <br>- Для игроков, оставшихся в игре: количество шагов в туториале <br>`== количество шагов в туториале`
6.4 | Метрики |
<tab> | MAE | `среднее абсолютных значений разностей между фактическим значением и предсказанным`
<tab> | Процент предсказаний в пределах <br>1 шага от фактического значения | 1. Разница между фактическим шагом отвала игрока и предсказанным значением <br>`фактическое значение - предсказание`: <br>- если `0 или 1` - успешное предсказание <br>- если `> 1 или < 1` - плохое предсказание <br>2. Вычисление процента <br>`(общее количество успешных предсказаний / общее количество предсказаний) * 100%`
6.6 | Подбор модели | С гиперпараметрическим поиском <br>- LinearRegression (baseline) <br>- DecisionTreeRegression <br>- RandomForestRegressor <br>- KNeighborsRegressor <br>- LGBMRegressor <br>- CatBoostRegressor
6.6 | Сравнение результатов | Выбор модели с оптимальными метриками
7 | **Тестирование модели** |

<br>

### 3.2 Как оценить качество полученной модели? Опишите методы которые знаете и какой считаете оптимальным и почему

Поскольку нужно восстановить зависимости между признаками и целевой переменной (что именно приводит к конкретному шагу, на котором игрок покидает туториал), нужно сравнить результаты предсказаний модели с фактическими значениями, известными при её построении. Для этого будут использованы offline-метрики, вычисленные по историческим данным.

* Целевая переменная - дискретное значение, конкретный шаг туториала (1, 2, 3, 4, 5 и т.д.)
* Задача машинного обучение - регрессия

Метрики регрессии:

Название | Сокращение | Описание | Отличительные особенности
-|-|-|-
Среднеквадратичная ошибка | $ MSE $ | Среднее квадратов разностей между фактическим значением и предсказанным | Квадратично штрафует за большие ошибки на объектах, но чувствительна к выбросам
Корень среднеквадратичной ошибки | $ RMSE $ | Квадратный корень из MSE | Неустойчива к выбросам
R-квадрат (коэффициент детерминации) | $ R^2 $ | Разность между единицей и отношением квадратов разностей между фактическим значением и предсказанным к квадратам разностей между фактическим значением и наилучшим константным предсказанием MSE (средним арифметическим предсказаний) | Указывает на соответствие модели данным, на которых построена
Средняя абсолютная ошибка | $ MAE $ | Среднее абсолютных значений разностей между фактическим значением и предсказанным | Устойчива к выбросам
Средняя абсолютная процентная ошибка | MAPE | Среднее отношения абсолютных разностей между фактическим значением и предсказанием к абсолютному фактическому значению | Выдаёт модели большой фиксированный штраф за ошибки, но чувствительна к прерывистым значениям
Симметричная средняя абсолютная процентная ошибка | SMAPE | Среднее отношения двухкратных абсолютных разностей между фактическим значением и предсказанием к суммам фактического значения с предсказанным | Выдаёт модели нулевой штраф
Взвешенная средняя процентная ошибка | WAPE | Отношение абсолютных разностей между фактическим значением и предсказанным к абсолютному фактическому значению | Устойчива к прерывистым значениям
Корень средней квадратичной логарифмической ошибки | RMSLE | Квадратный корень из среднего квадратов разностей между логарифмом суммы фактического значения и константы и логарифмом суммы предсказанного значения с константой | Вводит искусственную константу, чтобы избежать логарифма от нуля

Кастомные метрики применительно к конкретной поставленной перед исследованием задаче:

Название | Описание | Вычисления
-|-|-
Процент предсказаний в пределах 1 шага от фактического значения | 1. Разница между фактическим шагом отвала игрока и предсказанным значением <br><br>2. Вычисление процента | `фактическое значение - предсказание`: <br>- если `0 или 1` - успешное предсказание <br>- если `> 1 или < 1` - плохое предсказание <br><br>`(общее количество успешных предсказаний / общее количество предсказаний) * 100%`

><br>
><b>Вывод</b>
>
>В качестве оптимальных метрик в рамках поставленной задачи подходят:
>* MAE - стандартная метрика, которая поможет определить общую точность модели (чем меньше такая матрика, тем лучше и точнее модель предсказала ответы).
>* Кастомная метрика - вычисление процента предсказаний в пределах 1 шага от фактического значения поможет скорректировать точность предсказаний модели (чем больше такая метрика, тем лучше и точнее модель предсказала ответы).
><br><br>

<br>

### 3.3 Предположим что вы разработали модель которая с достаточной вероятностью предсказывает отвал. Опишите дизайн эксперимента и методологию по проверке гипотезы геймдизайнеров

><br>
><b>Обоснование</b>
>
>В качестве оптимального дизайна эксперимента подходит A/B-тестирование, поскольку оно позволяет понять, приведёт ли нововведение к улучшению результатов.
><br><br>

Дизайн эксперимента:
1. Разделить новых игроков на 2 группы случайным образом:
    * Контрольная группа - без изменений в ходе прохождения туториала
    * Экспериментальная группа - с запуском альтернативного прохождения туториала с шага, на котором разработанная модель предсказала выход из него игрока
2. Отслеживать изменение процента вышедших игроков в каждой группе
3. Сравнить процент вышедших игроков в обеих группах с учётом уровня статистической значимости ($ \alpha = 0.05 $):
    * Разница статистически значима - запуск альтернативного продолжения туториала с шага, на котором игрок покинет игру, увеличил прохождение туториала
    * Разница статистически не значима - запуск альтернативного продолжения туториала с шага, на котором игрок покинет игру, не увеличил прохождение туториала

Методология по проверке гипотезы геймдизайнеров:

№ | Этап | Описание
-|-|-
1 | **Определение минимального размера выборки** | Размер выборки каждой группы должен быть достаточным для обеспечения статистически значимых результатов
2 | **Сбор данных** |
<tab> | Группа | К какой группе принадлежит игрок (контрольная или экспериментальная)
<tab> | Шаг завершения туториала | На каком шаге туториала игрок ушёл из игры
<tab> | Состояние игрока | Завершил ли игрок туториал
3 | **Анализ данных** |
<tab> | Процент игроков, покинувших туториал | Получение процента игроков, ушедших из туториала, для каждой группы
<tab> | Статистический тест (z-тест) | Проверка значимости разницы процента отвала между контрольной и экспериментальной группами
4 | **Интерпретация результатов** |
<tab> | Отклонение нулевой гипотезы | Если результат z-теста показывает p-значение меньше уровня значимости ($ \alpha = 0.05 $), то нулевая гипотеза ($ H_0 $) отвергается - альтернативное продолжение туториала статистически значимо увеличивает прохождение туториала
<tab> | Подтверждение нулевой гипотезы | Если результат z-теста показывает p-значение больше уровня значимости ($ \alpha = 0.05 $), то нулевая гипотеза ($ H_0 $) не отвергается - альтернативное продолжение туториала не привело к статистически значимому улучшению прохождения туториала